In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Load CSV data from Part I into a dataframe
weather_df = pd.read_csv('output_data/cities.csv').drop(['Unnamed: 0'],axis=1)
weather_df

In [ ]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Pull data from main dataframe for heatmap layer
locations = weather_df[['Latitude','Longitude']].astype(float)
humidity_weight = weather_df['Humidity (%)']

fig=gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_weight,
                                 dissipating=False, max_intensity=100,
                                point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [3]:
# Create Dataframe fitting weather criteria
best_vacation = weather_df.loc[(weather_df['Temperature (F)'] <= 80) & (weather_df['Temperature (F)'] >= 70) & 
                               (weather_df['Wind Speed (mph)'] <= 10) & (weather_df['Cloudiness (%)'] == 0), :]

# Drop any rows with missing data
best_vacation.dropna(how='any')

,Cities,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
63,arraial do cabo,-22.9661,-42.0278,74.17,88,0,6.91
77,karratha,-20.7377,116.8463,75.47,59,0,7.83
120,beloha,-25.1667,45.0500,73.13,74,0,7.65
154,riyadh,24.6877,46.7219,79.93,19,0,2.30
224,cabedelo,-6.9811,-34.8339,78.80,89,0,9.22
227,vallenar,-28.5708,-70.7581,74.12,43,0,7.05
355,vila velha,-20.3297,-40.2925,78.80,83,0,5.75
441,ongandjera,-17.8833,15.0667,75.36,25,0,6.49
481,maceio,-9.6658,-35.7353,77.00,83,0,6.91
536,gat,31.6100,34.7642,73.67,36,0,9.95


In [24]:
# Hotel Map
hotel_df = weather_df.copy()
hotel_df

,Cities,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,caravelas,-17.7125,-39.2481,76.96,79,6,7.40
1,sitka,57.0531,-135.3300,57.20,33,20,3.44
2,tiksi,71.6872,128.8694,25.41,96,100,0.07
3,kaitangata,-46.2817,169.8464,44.01,63,84,4.00
4,bredasdorp,-34.5322,20.0403,55.40,94,100,4.61
...,...,...,...,...,...,...,...
553,chippewa falls,44.9369,-91.3929,44.82,31,90,8.05
554,ust-karsk,52.7011,118.8077,42.06,36,100,3.98
555,bardiyah,31.7561,25.0865,61.48,86,7,16.98
556,batagay,67.6333,134.6333,26.17,94,99,4.79


In [54]:
# geocoordinates
hotel_name = []
hotel_address = []
lats = hotel_df['Latitude']
lngs = hotel_df['Longitude']
x=0

for city in hotel_df['Cities']:
    target_coordinates = f'{lats[x]},{lngs[x]}'
    target_radius = 5000
    target_type = "lodging"
    x += 1

    # Set up a parameters dictionary
    params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }

    # Base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

    # Run a request using our params dictionary
    response = requests.get(base_url, params=params)

    temp = response.json()
    try:
        hotel_name.append(temp['results'][0]['name'])
        hotel_address.append(temp['results'][0]['vicinity'])
        print(f'Hotel in {city} found! Finding address...')
    except:
        hotel_name.append('N/A')
        hotel_address.append('N/A')
        print(f'No hotels in {city}.')
        
hotel_df['Hotel Name'] = hotel_name
hotel_df['Hotel Address'] = hotel_address

Hotel in caravelas found! Finding address...
Hotel in sitka found! Finding address...
Hotel in tiksi found! Finding address...
Hotel in kaitangata found! Finding address...
Hotel in bredasdorp found! Finding address...
Hotel in ponta do sol found! Finding address...
Hotel in ancud found! Finding address...
No hotels in saskylakh.
Hotel in port hardy found! Finding address...
Hotel in jiuquan found! Finding address...
Hotel in mataura found! Finding address...
Hotel in lebu found! Finding address...
No hotels in lavrentiya.
Hotel in san cristobal found! Finding address...
Hotel in qaanaaq found! Finding address...
Hotel in katsuura found! Finding address...
Hotel in coffs harbour found! Finding address...
No hotels in albany.
Hotel in nikolskoye found! Finding address...
Hotel in aktash found! Finding address...
Hotel in anadyr found! Finding address...
Hotel in hobart found! Finding address...
Hotel in mahebourg found! Finding address...
Hotel in nome found! Finding address...
Hotel in

Hotel in salalah found! Finding address...
Hotel in hobyo found! Finding address...
Hotel in saint george found! Finding address...
Hotel in kodiak found! Finding address...
Hotel in beringovskiy found! Finding address...
Hotel in hervey bay found! Finding address...
Hotel in roald found! Finding address...
Hotel in along found! Finding address...
Hotel in xiamen found! Finding address...
Hotel in victoria found! Finding address...
Hotel in vaini found! Finding address...
Hotel in edeia found! Finding address...
Hotel in vila do maio found! Finding address...
Hotel in chumikan found! Finding address...
Hotel in enshi found! Finding address...
Hotel in sur found! Finding address...
Hotel in mirzapur found! Finding address...
Hotel in ilebo found! Finding address...
Hotel in kalmar found! Finding address...
Hotel in margate found! Finding address...
No hotels in leningradskiy.
Hotel in dunedin found! Finding address...
Hotel in tabuk found! Finding address...
Hotel in bosilegrad found! F

Hotel in llandrindod wells found! Finding address...
Hotel in fereydun kenar found! Finding address...
Hotel in the valley found! Finding address...
Hotel in ngunguru found! Finding address...
Hotel in pimentel found! Finding address...
No hotels in ust-omchug.
Hotel in kjopsvik found! Finding address...
Hotel in hasaki found! Finding address...
Hotel in mehamn found! Finding address...
Hotel in moree found! Finding address...
Hotel in yulara found! Finding address...
Hotel in beidao found! Finding address...
No hotels in sobolevo.
Hotel in klaksvik found! Finding address...
Hotel in yelan-koleno found! Finding address...
Hotel in kieta found! Finding address...
No hotels in kedrovyy.
Hotel in codrington found! Finding address...
Hotel in iralaya found! Finding address...
No hotels in vostok.
Hotel in sao jose da coroa grande found! Finding address...
Hotel in lakes entrance found! Finding address...
Hotel in solnechnyy found! Finding address...
Hotel in bubaque found! Finding address.

In [55]:
hotel_df

,Cities,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name,Hotel Address
0,caravelas,-17.7125,-39.2481,76.96,79,6,7.40,Pousada dos Navegantes,"Rua das Palmeiras, 45, Caravelas"
1,sitka,57.0531,-135.3300,57.20,33,20,3.44,Westmark Sitka,"330 Seward Street, Sitka"
2,tiksi,71.6872,128.8694,25.41,96,100,0.07,Arktika,"Ulitsa Polyarnoy Aviatsii, д. 4, Tiksi"
3,kaitangata,-46.2817,169.8464,44.01,63,84,4.00,Kaitangata Motor Camp,"5 Clyde Terrace, Kaitangata"
4,bredasdorp,-34.5322,20.0403,55.40,94,100,4.61,Bredasdorp Country Manor,"9 Kloof Street, Bredasdorp"
...,...,...,...,...,...,...,...,...,...
553,chippewa falls,44.9369,-91.3929,44.82,31,90,8.05,AmericInn by Wyndham Chippewa Falls,"11 South Avenue, Chippewa Falls"
554,ust-karsk,52.7011,118.8077,42.06,36,100,3.98,N/A,N/A
555,bardiyah,31.7561,25.0865,61.48,86,7,16.98,بورتو بردية,"Unnamed Road, بردية،، بردية،"
556,batagay,67.6333,134.6333,26.17,94,99,4.79,Gostinitsa Ki'ileekh,"Ленина, 12а, Batagay"


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]